<a href="https://colab.research.google.com/github/innovativenexusbd/AgroAI/blob/main/disasteranalysisandprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing**

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

**Loading the Data, Preparing and Splitting the Data**

In [ ]:
disaster_df = pd.read_csv('disaster_data.csv')
X = disaster_df[['Year', 'Month', 'DisasterType', 'Occurrences']]
X = pd.get_dummies(X, columns=['DisasterType'], drop_first=True)  # Convert categorical variable to dummy/indicator variables
y_intensity = disaster_df['Intensity']
y_damage = disaster_df['CropDamage']
X_train, X_test, y_intensity_train, y_intensity_test, y_damage_train, y_damage_test = train_test_split(X, y_intensity, y_damage, test_size=0.2, random_state=42)


**Initializing Random Forest Regressor and Train the Model**

In [ ]:
intensity_model = RandomForestRegressor(random_state=42)
damage_model = RandomForestRegressor(random_state=42)
intensity_model.fit(X_train, y_intensity_train)
damage_model.fit(X_train, y_damage_train)


**Predicting the Future of the Disasters**

In [ ]:
future_years = np.arange(2024, 2026)
future_months = np.tile(np.arange(1, 13), len(future_years))
future_disasters = pd.DataFrame({
    'Year': np.repeat(future_years, 12),
    'Month': future_months,
    'DisasterType': np.random.choice(['Flood', 'Drought', 'Pest', 'Storm'], len(future_months)),
    'Occurrences': np.random.randint(0, 5, len(future_months))
})
future_X = pd.get_dummies(future_disasters, columns=['DisasterType'], drop_first=True)
missing_cols = set(X_train.columns) - set(future_X.columns)
for c in missing_cols:
    future_X[c] = 0
future_X = future_X[X_train.columns]
future_disasters['PredictedIntensity'] = intensity_model.predict(future_X)
future_disasters['PredictedDamage'] = damage_model.predict(future_X)

**Saving the Result**

In [ ]:
future_disasters.to_csv('future_disaster_analysis.csv', index=False)
print("Future Disaster Analysis:")
print(future_disasters.head(12))

**Visualizing the Results**

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(future_disasters['Month'], future_disasters['PredictedIntensity'], color='blue', marker='o')
plt.xlabel('Month')
plt.ylabel('Predicted Intensity')
plt.title('Predicted Disaster Intensity by Month')
plt.subplot(2, 1, 2)
plt.plot(future_disasters['Month'], future_disasters['PredictedDamage'], color='red', marker='o')
plt.xlabel('Month')
plt.ylabel('Predicted Crop Damage')
plt.title('Predicted Crop Damage by Month')
plt.tight_layout()
plt.show()